# Protein-Membrane MD Analysis tutorial using BioExcel Building Blocks (biobb)

***
This tutorial aims to illustrate the process of **analyzing a membrane molecular dynamics (MD) simulation** using the **BioExcel Building Blocks library (biobb)**. The particular example used is the heteropentameric ligand-gated chloride channel gated by gamma-aminobutyric acid (**GABA**), a major inhibitory neurotransmitter in the brain, which was embedded in a **DPPC membrane** in the [MemProtMD](https://memprotmd.bioch.ox.ac.uk/_ref/PDB/6i53/_sim/6i53_default_dppc/) project, which trajectory is obtained from the [**MDDB**](https://mddbr.eu/), where we can find it under the [A023K](https://irb-dev.mddbr.eu/#/id/A023K/overview) accession id.
***

## Settings

### Biobb modules used

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_mem](https://github.com/bioexcel/biobb_mem): Tools for membrane analysis and manipulation.
 - [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
 
### Auxiliary libraries used

* [jupyter](https://jupyter.org/): Free software, open standards, and web services for interactive computing across all programming languages.
* [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
* [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.
* [anywidget](https://anywidget.dev/en/getting-started/): Toolset for authoring reusable web-based widgets for interactive computing environments.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_mem.git
cd biobb_wf_mem
conda env create -f conda_env/environment.yml
conda activate biobb_wf_mem
jupyter-notebook biobb_wf_mem/notebooks/biobb_wf_mem.ipynb
```

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [Fetching Structure and Trajectory](#fetch)
 3. [Fitting](#fitting)
 4. [Membrane Identification with FATSLiM](#fatslim)
 5. [Z Positions and Membrane Thickness](#thickness)
 6. [Deuterium order parameter](#order)
 7. [Area per lipid](#apl)
 8. [Density Profile](#density)
 9. [Channel Dimensions](#channel)
 10. [Flip-flop rate](#flip-flop)
 11. [Questions & Comments](#questions)

***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***


## Initializing colab
The three cells below are used only in case this notebook is executed via **Google Colab**. Take into account that, for running conda on **Google Colab**, the **condacolab** library must be installed. As [explained here](https://pypi.org/project/condacolab/), the installation requires a **kernel restart**, so when running this notebook in **Google Colab**, don't run all cells until this **installation** is properly **finished** and the **kernel** has **restarted**.

In [ ]:
# Only executed when using google colab
import sys
if 'google.colab' in sys.modules:
  !git clone https://github.com/bioexcel/biobb_wf_mem.git
  !pip install -q condacolab
  import condacolab
  condacolab.install()

In [ ]:
if 'google.colab' in sys.modules: 
  # Install conda environment
  !mamba env update -n base -f biobb_wf_mem/conda_env/environment.yml
  # Fix Python path to prioritize conda packages
  conda_path = '/usr/local'
  sys.path.insert(0, f'{conda_path}/lib/python3.11/site-packages')
  # Enable widgets for colab
  from google.colab import output 
  output.enable_custom_widget_manager() 
  # Change working dir
  import os 
  os.chdir("biobb_wf_mem/biobb_wf_mem/notebooks") 
  print(f"New working directory: {os.getcwd()}")

In [ ]:
# Store original Colab paths
ORIGINAL_PATHS = [p for p in sys.path if '/usr/local/lib/python3.12' in p or 'dist-packages' in p]
CONDA_PATHS = [p for p in sys.path if '/usr/local/lib/python3.11' in p]

def use_colab_plotly():
    """Switch to Colab's original plotly"""
    # Move Colab paths to the front
    for path in reversed(ORIGINAL_PATHS):
        if path in sys.path:
            sys.path.remove(path)
        sys.path.insert(0, path)
    
    # Reload plotly and nbformat from Colab's environment
    if 'plotly' in sys.modules:
        del sys.modules['plotly']
        del sys.modules['plotly.graph_objs']
        # Delete all plotly submodules
        to_delete = [m for m in sys.modules if m.startswith('plotly')]
        for m in to_delete:
            del sys.modules[m]
    if 'nbformat' in sys.modules:
        del sys.modules['nbformat']

def use_conda_env():
    """Switch back to conda environment"""
    # Move conda paths to the front
    for path in reversed(CONDA_PATHS):
        if path in sys.path:
            sys.path.remove(path)
        sys.path.insert(0, path)
    
    # Clean up Colab's plotly/nbformat so conda versions load next time
    if 'plotly' in sys.modules:
        to_delete = [m for m in sys.modules if m.startswith('plotly')]
        for m in to_delete:
            del sys.modules[m]
    if 'nbformat' in sys.modules:
        del sys.modules['nbformat']

<a id="input"></a>
## Input parameters

**Input parameters** needed:
 - **MD_id**: MDDB id of the protein trajectory (e.g. [A023K](https://irb-dev.mddbr.eu/#/id/A023K/overview).
- **steps**: number of frames to skip during analysis.
 - **lipid_sel**: a [MDAnalysis selection](https://docs.mdanalysis.org/stable/documentation_pages/selections.html) of the membrane headgroups on the topology.
 - **ngl_sel**: and **ngl_hd_sel**: [NGLView](http://nglviewer.org/ngl/api/manual/selection-language.html) selection to visualize the membrane.
 - **cpptraj_sel**: [cpptraj](https://amberhub.chpc.utah.edu/cpptraj/) (Ambertools) selection to visualize the membrane.

In [ ]:
# Common imports
import os
import pandas as pd
import numpy as np
import nglview as nv
import MDAnalysis as mda
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from ipywidgets import HBox
from biobb_common import biobb_global_properties
# Set BioBB global properties
biobb_global_properties['can_write_file_log'] = False
biobb_global_properties['disable_logs'] = False

# All the inputs and outputs variables are also defined in the variables.py file
# in case you want to restart the notebook from another point
from variables import *

MD_id = 'A023K'
#MD_id = 'A01IP'
steps = 1
if MD_id == 'A023K':
    lipid_sel = '(resname DPPC and element P)'
    ngl_sel = 'DPPC'
    cpptraj_sel = 'DPP'
    ngl_hd_sel = f'{ngl_sel} and _P'
    frames = '1:-1:1000'
elif MD_id == 'A01IP':
    lipid_sel = '(resname POPC and element P)'
    ngl_sel = 'POPC'
    cpptraj_sel = 'POP'
    ngl_hd_sel = f'{ngl_sel} and _P'
    frames = '1:-1:4000'

# Common arguments for all the components
input_top_path = f'data/{MD_id}.pdb'
input_traj_path = f'data/{MD_id}.xtc'

For the **lipid_sel**, we only choose the element **P** of the residues with name **DPPC**, as this is the most charged one. Selecting only the **headgroups** make easier for the algorithms to separate the components of the **membrane**. 

<div style="text-align: center;">
    <img src="img/DPPC.png" alt="DPPC" width="800"/>
</div>

If you a different trajectory with a different or more lipids, make sure to adapt the selection using [MDAnalysis selection](https://docs.mdanalysis.org/stable/documentation_pages/selections.html) syntax.

***
<a id="fetch"></a>
## Fetching structure and trajectory

Download the structure and trajectory data, in **PDB** and **XTC** formats respectively, from the MD using the [MDDB API](https://mmb-dev.mddbr.eu/api/rest/docs/).
***
**Building Blocks** used:
 - [mddb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.mddb) from **biobb_io.api.mddb**
***

In [ ]:
# Check if folders 'out' and 'data' exist and create them if not
for folder in ['out', 'data']:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Check if PDB and XTC files already exist
pdb = f"data/{MD_id}_10.pdb"
xtc = f"data/{MD_id}_10.xtc"
tpr = f"data/{MD_id}.tpr"
if not os.path.exists(pdb) or not os.path.exists(xtc):

    # Download trajectory
    # Import module
    from biobb_io.api.mddb import mddb

    # Create properties dict and inputs/outputs
    prop = {
        'node_id' : 'irb-dev',
        'project_id': MD_id,
        'trj_format': 'xtc',
        'frames' : frames,
        'extra_files': [('topology.tpr',tpr)]
    }

    # Create and launch bb
    mddb(output_top_path=pdb,
         output_trj_path=xtc,
         properties=prop)

### Visualizing trajectory
Visualizing the downloaded **membrane-system trajectory** using **MDAnalysis** and **NGL**:  

In [ ]:
def show_membrane(universe):
    # Visualize the trajectory
    view_mem = nv.show_mdanalysis(universe)
    view_mem._remote_call('setSize', target='Widget', args=['400px','400px'])
    view_mem.clear_representations()
    view_mem.add_representation('cartoon', selection='all')
    
    # Visualize only the lipids (orange) with the headgroups highlighted (red)
    view_mem.add_representation('licorice', selection=ngl_sel, color='orange') # lipids
    view_mem.add_representation('spacefill', selection=ngl_hd_sel, color='red', radius=0.9) # headgroups
    view_mem.control.spin([1, 0, 0], -1.57)  # Align xy
    view_mem.camera='orthographic'
    return view_mem

In [ ]:
# Load the trajectory
u_prefit = mda.Universe(pdb, xtc)
show_membrane(u_prefit)

As you can see, the trajectory is fit to minimize the movement and rotation of the protein. In consecuence, the membrane seems to be moving a lot.

<a id="fitting"></a>
***
## Fitting the membrane

**Membrane analyses** typically assume that the **membrane** system is **aligned** with the **xy plane** and the **thickness** with the **z axis**.

<div style="text-align: center;">
    <img src="img/membrane_placement2.png" alt="Membrane Placement" width="400"></img>
</div>

In this example, **BioBB GROMACS** tools are used to properly **align** the system. 

***
**Building Blocks** used:
 - [GMXImage](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_image) from **biobb_analysis.gromacs.gmx_image** 
***

In [ ]:
# We fit the trajectory to the membrane so the z axis is aligned with the membrane
# Import module
from biobb_analysis.gromacs.gmx_image import gmx_image

# Create properties dict and inputs/outputs
memb_fit = f'data/{MD_id}_fit.xtc'

prop = {
    'fit_selection': 'DPPC',
    'output_selection': 'System',
    'center': False,
    'fit': 'rot+trans'
}

# Create and launch bb
gmx_image(input_traj_path=xtc,
          input_top_path=pdb,
          output_traj_path=memb_fit,
          properties=prop)

In [ ]:
# We fit again transxy to the protein so it does not shake due to lipids going through the pbc

memb_fit_xy = f'data/{MD_id}_fit2.xtc'

# Create properties dict and inputs/outputs
prop = {
    'fit_selection': 'Backbone',
    'output_selection': 'System',
    'center': False,
    'fit': 'transxy'
}

# Create and launch bb
gmx_image(input_traj_path=memb_fit,
            input_top_path=pdb,
            output_traj_path=memb_fit_xy,
            properties=prop)

In [ ]:
u = mda.Universe(pdb, memb_fit_xy)
# Trajectory before and after fitting to the membrane
HBox([show_membrane(u_prefit),show_membrane(u)])

<a id="fatslim"></a>
***
## Membrane identification with FATSLiM & Lipyphilic

**Membrane** and **leaflets** automatic identification.<br>
**Biological membranes** (e.g. cell membranes) are typically made up of a **phospholipid bilayer**. This **bilayer** consists of two layers (or **leaflets**) of **lipid molecules**:

- **Outer leaflet**: Faces the **extracellular environment** (outside the cell)

- **Inner leaflet**: Faces the **cytoplasm** (inside the cell)

Each **leaflet** can be composed of various **lipids**, **proteins**, and **carbohydrates**. <br>

**FATSLiM** and **Lipyphilic** tools help us to automatically identify the **leaflets**.
***
**Building Blocks** used:
 - [FatslimMembranes](https://biobb-mem.readthedocs.io/en/latest/fatslim.html#module-fatslim.fatslim_membranes) from **biobb_mem.fatslim.fatslim_membranes**
 - [LPPAssignLeaflets](https://biobb-mem.readthedocs.io/en/latest/lipyphilic_biobb.html#module-lipyphilic_biobb.lpp_assign_leaflets) from **biobb_mem.lipyphilic_biobb.lpp_assign_leaflets**
***

### FATSLiM leaflets identification
**FATSLiM** automatically identifies **membrane leaflets** and generates a **GROMACS** index file with this information.

In [ ]:
# Import module
# Analysis functionlipid_sel = '(resname DPPC and name P8)'
from biobb_mem.fatslim.fatslim_membranes import fatslim_membranes

# Create properties dict and inputs/outputs
leaflets_ndx = "out/leaflets.ndx"

prop = {
    'selection': lipid_sel,
    'cutoff': 2.2,
    'ignore_no_box': True
}

# Create and launch bb
fatslim_membranes(input_top_path=pdb,
                  #input_traj_path=memb_fit_xy,
                  input_traj_path=xtc,
                  output_ndx_path=leaflets_ndx,
                  properties=prop)

### Extract leaflets
The previous function generated a **GROMACS** index file (ndx). To extract the **lipids** in each **leaflet** from this file, we can use the <code>parse_index</code> auxiliary function.
These **atom indexes** can be used to select the **lipids** in the **membrane**. 

In [ ]:
from biobb_mem.fatslim.fatslim_membranes import parse_index

leaflets_dict = parse_index(leaflets_ndx)

for leaflet in leaflets_dict.keys():
    print(f"Leaflet found: {leaflet}")
    #print(f"Atoms:  {leaflets_dict[leaflet]}")

### Visualizing leaflets
Visualizing the automatically identified **leaflets**.<br>
Note that there is one **lipid** that is not in a **leaflet** (red coloured). This analysis can be then used as a **quality check** or in case we may want to ignore this **lipid** for the rest of the analyses.

In [ ]:
view = nv.show_structure_file(pdb)
view.clear_representations()
view.add_representation('cartoon', selection='all')
view.add_representation('licorice',selection=leaflets_dict['membrane_1_leaflet_1'], color='blue') # leaflet 1
view.add_representation('licorice',selection=leaflets_dict['membrane_1_leaflet_2'], color='cyan') # leaflet 2
view.add_representation('spacefill',selection=ngl_hd_sel, color='red', radius=0.9) # headgroups
#view.add_representation('spacefill',selection='2240', color='red') # lipid outlier
view.control.spin([1, 0, 0], -1.57)  # Align xy
view.layout.height = '500px'
view

### Lipyphilic leaflets identification
**Lipyphilic** assumes a **plane membrane**, is able to assign leafleats for **every frame** and use this information for **flip-flop** analysis.<br>

In [ ]:
# Import module
from biobb_mem.lipyphilic_biobb.lpp_assign_leaflets import lpp_assign_leaflets

# Create properties dict and inputs/outputs
prop = {
    'lipid_sel': lipid_sel,
    #'steps': 10,
    'steps': steps,
}

# Create and launch bb
lpp_assign_leaflets(input_top_path=pdb,
                    input_traj_path=memb_fit_xy,
                    output_leaflets_path= 'out/leaflets_data.csv',
                    properties=prop)

### Visualizing leaflets
Visualizing the automatically identified **leaflets**.<br>

In [ ]:
# Leaflets visualization
from biobb_mem.lipyphilic_biobb.lpp_assign_leaflets import display_nglview
view = display_nglview(pdb, 'out/leaflets_data.csv')
view.control.spin([0, 1, 0], -1.57)
view

<a id="thickness"></a>
***
## Z positions and membrane thickness

Calculating the **z distance** of **lipids** to the **bilayer center**.<br>
**Membrane thickness** and **local distortion** can be extracted from the **z-position** data. This information can also be used to compare **membrane thickness** in different regions, such as areas in contact with the **protein** versus the rest of the membrane.
***
**Building Blocks** used:
 - [LPPZPositions](https://biobb-mem.readthedocs.io/en/latest/lipyphilic_biobb.html#module-lipyphilic_biobb.lpp_zpositions) from **biobb_mem.lipyphilic_biobb.lpp_zpositions**
***

### Selecting protein-membrane region

Selecting the **membrane area** in contact with the **protein**.

In [ ]:
# We use MDAnalysis to select a region around the protein
# https://docs.mdanalysis.org/stable/documentation_pages/selections.html#geometric
around_sel = '(cyzone 50 70 -50 protein)'
around_resnums = u.select_atoms(around_sel).resnums
# Visualization of the selection in green
view = nv.show_mdanalysis(u)
view.clear_representations()
view.add_point(selection=ngl_sel, color='red')
view.add_point(selection=f'({", ".join(map(str, around_resnums))}) and not protein', color='green')
view.add_point(selection='protein', color='blue')
view

### Computing z-distances
Computing **z-positions** for both: **whole membrane** and the membrane region in contact with the **protein**

In [ ]:
# Run the analysis on the whole membrane
# Import module
from biobb_mem.lipyphilic_biobb.lpp_zpositions import lpp_zpositions

# Create properties dict and inputs/outputs
prop = {
    'lipid_sel': lipid_sel,
    #'steps': 1,
    'steps': steps,
    'height_sel': lipid_sel,
    'ignore_no_box': True
}

# Create and launch bb
lpp_zpositions(input_top_path=pdb,
               input_traj_path=memb_fit_xy,
               output_positions_path='out/zpositions.csv',
               properties=prop)

In [ ]:
# Run the analysis on the selection around the protein

# Create properties dict and inputs/outputs
prop = {
    'lipid_sel': lipid_sel,
    #'steps': 1,
    'steps': steps,
    'height_sel': f'{lipid_sel} and {around_sel}',
    'ignore_no_box': True
}

# Create and launch bb
lpp_zpositions(input_top_path=pdb,
            input_traj_path=memb_fit_xy,
            output_positions_path='out/zpositions_around.csv',
            properties=prop)

### Plotting the thickness of the membrane
Mean **z-position** values for the **positive** (upper leaflet) and **negative** (bottom leaflet) sides are plotted, along with the **membrane thickness** calculated as their difference. Solid lines represent the **entire membrane**, while dashed lines indicate the region in contact with the **protein**. The **standard deviation** of the **thickness** is shown as a grey shaded area.

In [ ]:
from biobb_mem.lipyphilic_biobb.lpp_zpositions import frame_df

grouped = frame_df('out/zpositions.csv')
around = frame_df('out/zpositions_around.csv')
plt.figure(figsize=(10, 6))
# Plot the mean positive and mean negative z positions
plt.plot(grouped['mean_positive'], label='Mean Positive', linestyle='-', color='blue')
plt.plot(around['mean_positive'], label='Mean Positive 5A around protein', linestyle='--', color='blue')
plt.plot(grouped['mean_negative'], label='Mean Negative', linestyle='-', color='red')
plt.plot(around['mean_negative'], label='Mean Negative 5A around protein', linestyle='--', color='red')
plt.plot(grouped['thickness'], label='Thickness', linestyle='-', color='green')
plt.plot(around['thickness'], label='Thickness 5A around protein', linestyle='--', color='green')
plt.fill_between(grouped.index, grouped['thickness'] - grouped['std_thickness'], grouped['thickness'] + grouped['std_thickness'], color='green', alpha=0.2, label='Thickness ± Std')
plt.title('Scatter Plot of Mean Positive and Mean Negative Z Positions')
plt.ylabel('Z Position (Å)'); plt.xlabel('Frame')
plt.legend(loc='center left', bbox_to_anchor=(1., 0.5), ncol=1, fancybox=True, shadow=True)
plt.show()


### Representing and animating the local distorsion

The **z-positions** are used to create an **animation** illustrating **lipid displacements** from the **mean distance** for both the **upper** and **bottom leaflets** of the **membrane bilayer**. **Lipids** that are **below** the **mean distance** are colored in **blue**, while those **above** the **mean** are colored in **red**.

In [ ]:
# Custom coloring scheme for the lipids with JavaScript
# https://projects.volkamerlab.org/teachopencadd/talktorials/T017_advanced_nglview_usage.html#Custom-coloring-schemes-and-representations
template = """
this.atomColor = function(atom) {
    let zPos = atom.z - %(midplane_z)f;
    let std = %(factor)f * %(std)f;
    let ratio, r, g, b, layerZ;
    
    if (zPos > 0) { // top leaflet
        layerZ = %(top_z)f;
    } else { // bottom leaflet
        layerZ = -1 * %(bot_z)f;
        zPos = -zPos;
    }
    
    if (zPos >= layerZ) { // thickening
        ratio = (zPos - layerZ) / std;
        ratio = Math.min(1, ratio);
        r = Math.floor(255 * (1 - ratio));
        g = Math.floor(255 * (1 - ratio)); 
        b = 255;
    } else { // thinning
        ratio = -(zPos - layerZ) / std;
        ratio = Math.min(1, ratio);
        r = 255;
        g = Math.floor(255 * (1 - ratio));
        b = Math.floor(255 * (1 - ratio));
    }
    return (r << 16) + (g << 8) + b;
}
"""

# Calculate the mean positive and negative (top and bottom leaflets) z positions 
# wrt the midplane of the membrane, and the standard deviation of the thickness.
top_z = grouped['mean_positive'].mean()
bot_z = grouped['mean_negative'].mean()
std   = grouped['std_thickness'].mean()
top_z, bot_z, std

# Calculate the mean z position of midplane wrt the box axis.
midplane_z = [u.trajectory[i].positions[u.select_atoms(lipid_sel).indices].mean(axis=0)[2] for i in range(0, u.trajectory.n_frames, steps)]
def on_frame_change(change):
    frame = change['new']
    # We use the midplane  
    subframe = frame // steps
    js_function_combined = template % {
        'factor': 3, # Factor to increase the color fade
        'midplane_z': midplane_z[subframe],
        'std': std,
        'top_z': top_z,
        'bot_z': bot_z
    }
    # We have to change the color registry name to trigger the update
    nm = "local_dist"+str(frame) # One color per frame
    nv.color.ColormakerRegistry.add_scheme_func(nm, js_function_combined)
    view.update_ball_and_stick(color=nm)

In [ ]:
# Play the animation to see the color change
# RED: Thinning, BLUE: Thickening

view = nv.show_mdanalysis(u)
view.clear_representations()
view.add_ball_and_stick(selection=ngl_hd_sel,aspect_ratio=8)
view.add_cartoon(selection='protein', color='grey')
view.layout.height = '500px'
view.background = 'grey'

view.observe(on_frame_change, names=['frame'])
view

<a id="order"></a>
***
## Deuterium order parameter

Calculating deuterium order parameter of acyl tails in a lipid bilayer using [gorder](https://github.com/Ladme/gorder). <br>
***
**Building Blocks** used:
 - [GOrderAA](https://biobb-mem.readthedocs.io/en/latest/gorder.html#module-gorder.gorder_aa) from **from biobb_mem.gorder.gorder_aa**
***

In [ ]:
from biobb_mem.gorder.gorder_aa import gorder_aa

prop = {
    'handle_pbc': False
}

gorder_aa(input_top_path=tpr,
          input_traj_path=xtc,
          output_order_path='out/order.csv',
          properties=prop)

In [ ]:
# Read and process the order parameter data
df_order = pd.read_csv('out/order.csv')

# Separate data for sn-1 (C2X) and sn-2 (C3X) chains
# Separate data for sn-1 (C2X) and sn-2 (C3X) chains
sn1_data = df_order[df_order['atom'].str.startswith('C2')].copy().reset_index()
sn2_data = df_order[df_order['atom'].str.startswith('C3')].copy().reset_index()

fig, ax = plt.subplots(figsize=(8,6))
ax.plot(sn1_data['total'], marker='o', label='sn-1 (C2X)', color='blue')
ax.plot(sn2_data['total'], marker='s', label='sn-2 (C3X)', color='orange')
ax.set_xlabel('Carbon Position')
ax.set_ylabel('Order Parameter (-SCD)')
ax.set_title('DPPC Deuterium Order Parameters - Both Acyl Chains')
ax.grid(True, linestyle='--', alpha=0.5)
ax.legend()
plt.tight_layout()
plt.show()

<a id="apl"></a>
***
## Area per lipid

Area per lipid via a 2D Voronoi tessellation.<br>
While diving the space for the lipids we remove the area correspoding to protein molecules.
***
**Building Blocks** used:
 - [FatslimAPL](https://biobb-mem.readthedocs.io/en/latest/fatslim.html#module-fatslim.fatslim_apl) from **biobb_mem.fatslim.fatslim_apl**
***

In [ ]:
from biobb_mem.fatslim.fatslim_apl import fatslim_apl

prop = {
    'lipid_selection': lipid_sel,
    'protein_selection': 'protein and not element H',
    'ignore_no_box': True,
}
fatslim_apl(input_top_path=pdb,
            input_traj_path=memb_fit,
            output_csv_path='out/apl.csv',
            properties=prop)

In [ ]:
# Load the data
df = pd.read_csv('out/apl.csv')

# Scatter plot of spatial distribution
plt.figure(figsize=(8,6))
plt.scatter(df['X coords'], df['Y coords'], c=df['Area per lipid'], cmap='viridis')
plt.colorbar(label='Area per lipid')
plt.xlabel('X coordinates'); plt.ylabel('Y coordinates')
plt.title('Spatial Distribution of Area per Lipid')
plt.show()

In [ ]:
from scipy.interpolate import griddata

def plot_apl(output_csv_path, hist=False, res=70j):
    """Create a 2D visualization of the area per lipid for both membrane leaflets.

    This function reads area per lipid data from a CSV file, creates interpolated 
    2D maps for both upper and lower leaflets, and displays them side-by-side.

    Parameters
    ----------
    output_csv_path : str
        Path to the CSV file containing area per lipid data
    hist : bool, optional
        If True, use imshow for visualization, otherwise use contourf (default False)
    res : complex, optional
        Grid resolution for interpolation (default 70j)

    Returns
    -------
    list
        List of interpolated grids for upper and lower leaflets
    """
    df = pd.read_csv(output_csv_path)
    grids = []
    # Create separate plots for each leaflet
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5), sharey=True, sharex=True)
    m = df['Area per lipid'].median()
    s = df['Area per lipid'].std()
    vmin, vmax = max(0, m-s), m+s
    # Define common grid for both plots
    x_all = df['X coords']
    y_all = df['Y coords']
    grid_x, grid_y = np.mgrid[min(x_all):max(x_all):res,
                             min(y_all):max(y_all):res]
    for ax, leaflet in zip([ax1, ax2], ['upper leaflet','lower leaflet']):
        df_leaflet = df[df['leaflet'] == leaflet]
        points = np.stack((np.array(df_leaflet['X coords']).T, np.array(df_leaflet['Y coords']).T), axis=-1)
        values = np.array(df_leaflet['Area per lipid'])
        grid = griddata(points, values, (grid_x, grid_y), method='cubic')
        grids.append(grid)
        # Plot map
        if hist:
            im = ax.imshow(grid, extent=[min(x_all), max(x_all), min(y_all), max(y_all)], 
                  origin='lower', vmin=vmin, vmax=vmax, cmap='coolwarm_r')
        else:
            im = ax.contourf(grid_x, grid_y, grid, vmin=vmin, vmax=vmax, cmap='coolwarm_r')
        plt.colorbar(im, ax=ax)
        ax.set_title(f'Area per lipid - {leaflet}')
        ax.set_xlabel("Box X (nm)")
        ax.set_ylabel("Box Y (nm)" if ax == ax1 else "")
    plt.tight_layout()
    return grids

In [ ]:
grid = plot_apl('out/apl.csv', hist=True)

<a id="density"></a>
***
## Density profile

**Membrane density** refers to the spatial distribution of atoms, molecules, or mass within a **lipid bilayer** typically measured along the **membrane normal** (usually the **z-axis**).

**Membrane density** is calculated over the **z-axis** here with **cpptraj** tool from **AmberTools** and using the [AMBER selection masks](https://amberhub.chpc.utah.edu/atom-mask-selection-syntax/).<br>
***
**Building Blocks** used:
 - [CpptrajDensity](https://biobb-mem.readthedocs.io/en/latest/ambertools.html#module-ambertools.cpptraj_density) from **biobb_mem.ambertools.cpptraj_density**
***

In [ ]:
# Import module
from biobb_mem.ambertools.cpptraj_density import cpptraj_density

cpptraj_mask = f":{cpptraj_sel} !:{cpptraj_sel} :{cpptraj_sel}@P8"
print(f"CPPTRAJ mask for {{membrane - non-membrane - headgroups}}: {cpptraj_mask}")

# Create properties dict and inputs/outputs
prop = {
    'mask': cpptraj_mask,
    #'steps': 1,
    'steps': steps
}

# Create and launch bb
cpptraj_density(input_top_path=pdb,
                input_traj_path=memb_fit_xy,
                output_cpptraj_path='out/density.dat', 
                properties=prop)

### Visualizing system densities

Visualizing the **protein**, **membrane** and **P8 densities** for the system along the **z-axis**. The corresponding molecular structure is displayed using **NGL** in the same orientation as the 2D-plot, allowing for direct spatial comparison between the **structural** and **density** representations.<br>

In [ ]:
# NGL viewer
view = nv.show_file(pdb)
df = pd.read_csv(f'out/density.dat', sep='\s+')
labels = ['Membrane','Protein','P8']

# Plotly 2D plot
axis = df.columns[0] # Axis the density was calculated on
cls = df.columns[1:] # Masks used 
fig = make_subplots(rows=1, cols=1, subplot_titles=['Density along z-axis'])

# Iterate over the columns
for i in range(0,len(cls),2): # Density of each mask
    fig.add_trace(go.Scatter(x=df[axis], y=df[cls[i]], mode='lines', name=labels[i // 2]), row=1, col=1)

# Standard deviation
for i in range(1,len(cls)+1,2): 
    # Add a transparent line plot for the standard deviation
    fig.add_trace(go.Scatter(x=df[axis], y=df[cls[i-1]] - df[cls[i]], mode='lines', fill=None, showlegend=False, line=dict(color='rgba(0,0,0,0)')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df[axis], y=df[cls[i-1]] + df[cls[i]], mode='lines', fill='tonexty', name=f'{labels[i // 2]} STD', line=dict(color='rgba(0,0,0,0)'), fillcolor='rgba(0,100,80,0.2)'), row=1, col=1)
    
fig.update_layout(
    xaxis_title='Å', 
    yaxis_title='Density', 
    legend=dict(orientation="h",
        yanchor="bottom", y=-0.5, xanchor="auto",
        entrywidth=0.33, entrywidthmode='fraction',
        traceorder="normal"),
    height=400, # Make figure smaller by setting fixed height
    margin=dict(l=40, r=40, t=40, b=40)
)
display(fig)
# Set the size of the NGL view widget and its orientation
view.remove_ball_and_stick()
view.update_cartoon(color='red')
view.add_point(selection=ngl_sel, color='blue')
view.add_ball_and_stick(selection=ngl_hd_sel, color='green',aspect_ratio=4) # headgroups
view.background = 'black'
view.layout.width = '100%'
display(view)
view.control.spin([0, 1, 0], -1.57)  # Same orientation as the plot
#view._remote_call('setSize', target='Widget', args=['100%', '100%'])

<a id="channel"></a>
***
## Channel dimensions

A **membrane channel** or **pore** is a **passage** or **opening** formed by a **protein structure** embedded in a **biological membrane**, allowing the **selective movement** of ions, molecules, or other substances across the **membrane**. **Channels** are typically **gated**, meaning they can **open** or **close** in response to specific **signals** (such as changes in voltage, ligand binding, or mechanical stress).

**MDanalysis hole** tool can be used to identify **channels** and analyze **dimensions** and **properties** along them.<br>
***
**Building Blocks** used:
 - [MDAHole](https://biobb-mem.readthedocs.io/en/latest/mdanalysis_biobb.html#mdanalysis_biobb.mda_hole.MDAHole) from **from biobb_mem.mdanalysis_biobb.mda_hole** 
***

In [ ]:
# Import module
from biobb_mem.mdanalysis_biobb.mda_hole import mda_hole

# Create properties dict and inputs/outputs
prop = {
    'select': 'protein',
    #'steps': steps,
    'steps': 20,
    'sample': 0.1,
}

# Create and launch bb
mda_hole(input_top_path=pdb,
         input_traj_path=memb_fit_xy,
         #input_traj_path=xtc,
         output_hole_path='out/hole.vmd',
         output_csv_path='out/hole_profile.csv',
         properties=prop)

### Plotting channel dimensions

A **2D plot** representing the **pore radius** along the **pore coordinate**, with the bottom representing the **cytoplasm** and the top representing the **extracellular** space.

In [ ]:
# Load the CSV file
df = pd.read_csv('out/hole_profile.csv')
frames = df['Frame'].unique()

# Create a figure
fig = go.Figure()

# Add traces for each frame
for frame in frames:
    frame_data = df[df['Frame'] == frame]
    fig.add_trace(go.Scatter(x=frame_data['Pore Coordinate'], y=frame_data['Radius'], mode='lines', name=f'Frame {frame}'))

# Update layout
fig.update_layout(
    title='Pore Radius vs Pore Coordinate',
    xaxis_title='Pore Coordinate (Å)',
    yaxis_title='Radius (Å)',
    legend_title='Frame'
)

# Show the figure
fig.show()

### Visualizing channel

In [ ]:
from biobb_mem.mdanalysis_biobb.mda_hole import display_hole
display_hole(pdb, memb_fit_xy, output_hole_path='out/hole.vmd', opacity=0.9,frame=0)

<a id="flip-flop"></a>
***
## Flip-flop rate

This module provides methods for detecting the **flip-flop** of molecules in a **lipid bilayer**.<br>
A **flip-flop** occurs when a molecule (typically a sterol) moves from one **leaflet** of a **bilayer** into the **opposing leaflet**. This movement can occur spontaneously, although it is generally slow due to the **energy barrier** involved in crossing the **hydrophobic** interior of the **membrane**.<br>
***
**Building Blocks** used:
 - [LPPFlipFlop](https://biobb-mem.readthedocs.io/en/latest/lipyphilic_biobb.html#module-lipyphilic_biobb.lpp_flip_flop) from **from biobb_mem.lipyphilic_biobb.lpp_flip_flop**
***

In [ ]:
# Import module
from biobb_mem.lipyphilic_biobb.lpp_flip_flop import lpp_flip_flop

# Create properties dict and inputs/outputs
prop = {
    'lipid_sel': lipid_sel,
    #'steps': 10,
    'steps': steps,
    'frame_cutoff': 2
}

# Create and launch bb
lpp_flip_flop(input_top_path=pdb,
              input_traj_path=memb_fit_xy,
              input_leaflets_path='out/leaflets_data.csv',
              output_flip_flop_path='out/flip_flop.csv',
              properties=prop)

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)
    * [https://github.com/bioexcel/biobb_mem](https://github.com/bioexcel/biobb_mem)
    * [https://github.com/bioexcel/biobb_wf_mem/issues](https://github.com/bioexcel/biobb_wf_mem/issues)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)

* Mail:
    * [ruben.chaves@irbbarcelona.org](mailto:ruben.chaves@irbbarcelona.org)